In [ ]:
from pathlib import Path
from collections import defaultdict

In [ ]:
outdir_validation_set = Path("/home/lien002/nobackup_lien002/projects/scs/antismash_dataset_scs/output/output_ipresto_main_set")
input_file = outdir_main_set / "presto_top" / "matches_per_topic_filtered.txt"

In [ ]:
def get_tokenised_genes_from_match_line(match_line):
    tokenised_genes = list()
    match = match_line.split()[2]
    for gene_and_topic_prob in match.split(","):
        gene, gene_to_topic_prob = gene_and_topic_prob.split(":")
        tokenised_genes.append(gene)
    return tokenised_genes

In [ ]:
subcluster_matches = defaultdict(list)
with open(input_file, "r") as infile:
    for line in infile: 
        # skip header lines
        if line.startswith("#"): 
            continue
        tokenised_genes = get_tokenised_genes_from_match_line(line)
        if len(tokenised_genes) < 2:
            # skip matches with less than 2 (unique) genes
            continue
        bgc_id = line.split()[3]
        subcluster_matches[tokenised_genes].append(bgc_id)

In [ ]:
top_modules = list(subcluster_matches.keys())

In [ ]:
I am writing code in python and have a list of sets called top_modules.

use k_means clustering to cluster the sets. First transform it into a sparse binary matrix

In [ ]:
def cluster_kmeans(
    sparse_m, modules, num_clusters, rownames, colnames, prefix, header, cores=1
):
    """Kmeans clustering on sparse_m with num_clusters and writes to file

    sparse_m: csr_matrix, shape(n_samples, n_features)
    modules: dict {mod_num:[info,modules]}
    num_clusters: int, number of clusters
    rownames: list of ints, [mod_nums], sequential mod_nums, keeping track of
        rows of sparse_m
    colnames: list of str, all domains sequential order to keep track of
        columns of sparse_m
    prefix: str, prefix of outfile
    cores: int, amount of cores to use
    header: str, header of module file
    """
    print("\nRunning k-means")
    # outfiles
    kmeans_pre = f"_{num_clusters}_families"
    out_mods = prefix + kmeans_pre + ".txt"
    out_clusts = prefix + kmeans_pre + "_by_family.txt"

    # running algorithm
    kmeans = KMeans(
        n_clusters=num_clusters,
        n_init=20,
        max_iter=1000,
        random_state=595,
        verbose=0,
        tol=0.000001,
        n_jobs=cores,
    ).fit(sparse_m)
    print(kmeans)
    clust_centers = sp.csr_matrix(kmeans.cluster_centers_)
    labels = kmeans.labels_
    cluster_dict = defaultdict(list)
    np.set_printoptions(precision=2)
    print("Within-cluster sum-of-squares (inertia):", kmeans.inertia_)
    # link each module to a family/k-means cluster
    with open(out_mods, "w") as outf:
        outf.write(header + "\tFamily\n")
        for subcl, cl in zip(rownames, labels):
            cluster_dict[cl].append(subcl)
            outf.write(
                "{}\t{}\t{}\n".format(subcl, "\t".join(modules[subcl]), cl)
            )
    # write file of listing all families/clusters by family with their modules
    avg_clst_size = []
    with open(out_clusts, "w") as outf_c:
        for i in range(clust_centers.shape[0]):
            matches = cluster_dict[i]
            l_matches = len(matches)
            avg_clst_size.append(l_matches)
            counts = Counter(
                [dom for m in matches for dom in modules[m][-1].split(",")]
            )
            spars = clust_centers[i]
            feat_inds = spars.nonzero()[1]
            feat_tups = [(spars[0, ind], colnames[ind]) for ind in feat_inds]
            feat_format = [
                "{}:{:.2f}".format(dom, float(score))
                for score, dom in sorted(feat_tups, reverse=True)
            ]
            outf_c.write(
                "#Subcluster-family {}, {} subclusters\n".format(i, l_matches)
            )
            outf_c.write(
                "#Occurrences: {}\n".format(
                    ", ".join(
                        [dom + ":" + str(c) for dom, c in counts.most_common()]
                    )
                )
            )
            outf_c.write("#Features: {}\n".format(", ".join(feat_format)))
            # maybe as a score the distance to the cluster center?
            for match in matches:
                outf_c.write(
                    "{}\t{}\n".format(match, "\t".join(modules[match]))
                )
    print("\nAverage clustersize:", np.mean(avg_clst_size))